In [1]:
import torch
import numpy as np
from torch import nn
import random
import torch.nn.functional as F
import collections
from torch.optim.lr_scheduler import StepLR
from CommunicationP3DX import CommunicationP3DX
from Agent import AgentClass
from std_srvs.srv import Empty
import rospy
import time
import shelve
import os
from datetime import datetime

workspace = shelve.open('../SOGOODWTF_wsh_mean4_20230929_203934/wsh_20230929_203934.out')
         # true if the key exists
vars = list(workspace.keys())
print(workspace)
print(vars)

['Agent', 'AgentClass', 'CommunicationP3DX', 'Empty', 'Memory', 'QNetwork', 'QNetworkCNN', 'Q_1', 'StepLR', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '__warningregistry__', 'actions_angular', 'actions_linear', 'datetime', 'device', 'evaluate', 'filename', 'goal_zones_x', 'goal_zones_y', 'hist_dict', 'laser_scan_state_type_atual', 'main', 'max_episodes', 'my_shelf', 'n_sectors', 'path', 'pause_physics_client', 'performance', 'reset_simulation', 'select_action', 'str_hora_inicio_treino', 'theta_atual', 'train', 'unpause_physics_client', 'update_parameters']


In [2]:
hist_dict = workspace['hist_dict']
print(hist_dict.keys())

for key in hist_dict.keys():
    print('key: '+str(key)+'\t type> '+str(type(hist_dict[key])))

dict_keys(['pos', 'scan', 'rewards', 'rates', 'state'])
key: pos	 type> <class 'dict'>
key: scan	 type> <class 'dict'>
key: rewards	 type> <class 'dict'>
key: rates	 type> <class 'dict'>
key: state	 type> <class 'dict'>


In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

for episode in hist_dict['rewards'].keys():
    print('Ep: '+str(episode)+' - rewards \t> ' +str(hist_dict['rewards'][episode]))

print('\n')

for episode in hist_dict['rates'].keys():
    print('Ep: '+str(episode)+' - [eps, lr] \t> ' +str(hist_dict['rates'][episode]))

Ep: 101 - rewards 	> 2635.234337841574
Ep: 201 - rewards 	> 688.2658740966046


Ep: 101 - [eps, lr] 	> [0.6658522711696339, 0.0009363292088239415]
Ep: 201 - [eps, lr] 	> [0.6333703528882284, 0.0009060305822453377]


In [4]:
n_eps = len(hist_dict['scan'].keys())

print(hist_dict['scan'][150][5])

print(hist_dict['state'][150][5])

(2.1462764739990234, 2.1871137619018555, 2.1993954181671143, 2.21269154548645, 2.205157518386841, 2.22029709815979, 2.2521016597747803, 2.2402021884918213, 2.25927996635437, 2.278407335281372, 2.2760813236236572, 2.27386212348938, 2.3109848499298096, 2.309422731399536, 2.3404269218444824, 2.3491640090942383, 2.3576338291168213, 2.3756167888641357, 2.3757588863372803, 2.3805928230285645, 2.427170753479004, 2.4231348037719727, 2.4356582164764404, 2.4581925868988037, 2.4662671089172363, 2.4720680713653564, 2.492699384689331, 2.5038228034973145, 2.5394744873046875, 2.54254412651062, 2.5346059799194336, 2.5676827430725098, 2.5906872749328613, 2.6069769859313965, 2.628777503967285, 2.6456093788146973, 2.6556787490844727, 2.6659069061279297, 2.707406997680664, 2.717376470565796, 2.741013288497925, 2.7045905590057373, 2.668006181716919, 2.680628776550293, 2.6769611835479736, 2.6476056575775146, 2.641611099243164, 2.635037660598755, 2.6258912086486816, 2.601863384246826, 2.610537052154541, 2.58

In [5]:
type(hist_dict['state'][episode])

list

In [6]:
episode = 150 
for iteration,state in enumerate(hist_dict['state'][episode]):
    # hist_dict['state'][episode][iteration]
    print('i: '+str(iteration)+' > '+str(state))

i: 0 > [0, 0, 0, 0, 5]
i: 1 > [0, 0, 1, 0, 5]
i: 2 > [1, 2, 1, 0, 5]
i: 3 > [2, 0, 0, 0, 5]
i: 4 > [0, 0, 0, 0, 4]
i: 5 > [0, 0, 0, 0, 4]
i: 6 > [0, 1, 0, 0, 2]
i: 7 > [2, 2, 1, 0, 3]


In [7]:
from numpy.random import randint

rdn_eps = randint(0, 300, 5)
print(rdn_eps)
for episode in rdn_eps:
    print('\n e:'+str(episode))
    for iteration,state in enumerate(hist_dict['state'][episode]):
        print('i: '+str(iteration)+' \t> '+str(state))

[165  99  97  14  66]

 e:165
i: 0 	> [0, 0, 0, 0, 5]
i: 1 	> [0, 1, 0, 0, 4]
i: 2 	> [2, 0, 0, 0, 4]
i: 3 	> [0, 0, 0, 0, 2]
i: 4 	> [0, 0, 1, 0, 4]
i: 5 	> [0, 1, 0, 0, 5]
i: 6 	> [2, 2, 0, 0, 5]

 e:99
i: 0 	> [0, 0, 0, 0, 5]
i: 1 	> [0, 0, 0, 0, 5]
i: 2 	> [0, 0, 0, 0, 4]
i: 3 	> [2, 0, 0, 0, 2]
i: 4 	> [0, 0, 0, 0, 2]
i: 5 	> [0, 0, 0, 0, 4]
i: 6 	> [0, 0, 0, 0, 3]
i: 7 	> [0, 0, 0, 0, 5]
i: 8 	> [0, 0, 0, 1, 4]
i: 9 	> [0, 2, 2, 2, 3]

 e:97
i: 0 	> [0, 0, 0, 0, 5]
i: 1 	> [0, 0, 1, 2, 5]
i: 2 	> [0, 0, 0, 0, 5]
i: 3 	> [0, 0, 0, 0, 5]
i: 4 	> [0, 1, 1, 0, 5]
i: 5 	> [1, 1, 0, 0, 5]
i: 6 	> [1, 0, 0, 0, 5]
i: 7 	> [0, 0, 0, 0, 3]
i: 8 	> [0, 0, 1, 0, 3]
i: 9 	> [0, 2, 0, 0, 2]
i: 10 	> [2, 2, 0, 0, 2]

 e:14
i: 0 	> [0, 0, 0, 0, 5]
i: 1 	> [1, 0, 0, 0, 5]
i: 2 	> [0, 0, 0, 0, 5]
i: 3 	> [0, 0, 0, 0, 3]
i: 4 	> [0, 0, 2, 1, 3]
i: 5 	> [0, 0, 1, 2, 3]

 e:66
i: 0 	> [0, 0, 0, 0, 5]
i: 1 	> [0, 0, 1, 1, 5]
i: 2 	> [2, 1, 1, 2, 4]


In [8]:
agente = AgentClass()
done,status_done = agente.is_done(300)
print(done)
print(status_done)

300
True
2


In [9]:
i=400
horizon = 500# np.inf
agente = AgentClass()
done =False

while not done:
    i = i+ 1
    #if state is None: break
    #action = select_action(Q_2, action_space_size, state, eps)

    #TODO take action here, calculate rewards, check if done, refresh state variable
    #state, reward, done, _ = env.step(action)

    # if action==0:
    #     Com.action_forward()
    # elif action==1:
    #     Com.action_right()
    # else:
    #     Com.action_left()
    if i%1000==0:print(i)

    # state  = Agent.get_state_discrete(laser_scan_state_type=laser_scan_state_type_atual, theta=theta_atual)
    # #print(state)
    # reward = Agent.get_reward()
    done,status_done = agente.is_done(number_iterations=i,max_iterations=horizon, reach_dist=0.5)

    # # hist_dict = {'pos':{}, 'scan':{}, 'rewards':{}, 'rates':{}, 'state':{}}
    # if i==1:
    #     hist_dict['pos'][episode+1] = [Agent.Pos]
    #     hist_dict['scan'][episode+1] = [Agent.laser_scan]
    #     hist_dict['state'][episode+1] = [state]
    # else:
    #     hist_dict['pos'][episode+1].append(Agent.Pos)
    #     hist_dict['scan'][episode+1].append(Agent.laser_scan)
    #     hist_dict['state'][episode+1].append(state)

    if i > horizon:
        print('h')
        done = True

    # render the environment if render == True
    #if render and episode % render_step == 0:
    #    env.render()

    # save state, action, reward sequence
    #memory.update(state, action, reward, done)
print(i)
print(status_done)
print('out')
#1 for not done, 0 for crash, 1 for goal 2 for time

501
h
501
2
out


In [10]:
hist_dict = {'pos':{}, 'scan':{}, 'rewards':{}, 'rates':{}, 'state':{}, 'epresult':{}}
episode = 0
hist_dict['epresult'][episode+1] = [done, status_done]
hist_dict

{'pos': {},
 'scan': {},
 'rewards': {},
 'rates': {},
 'state': {},
 'epresult': {1: [True, 2]}}

In [11]:
episode = 1
hist_dict['epresult'][episode+1] = [False, 0]
hist_dict

{'pos': {},
 'scan': {},
 'rewards': {},
 'rates': {},
 'state': {},
 'epresult': {1: [True, 2], 2: [False, 0]}}